In [1]:
import xarray as xr
import numpy as np
import glob as glob
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
import cartopy.feature as cfeature
from matplotlib.animation import FuncAnimation
import imageio.v2 as imageio
import numpy as np
import sparse

In [2]:
# List of file paths
file_paths = [
"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2080*.nc",
    "/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2081*.nc",
    "/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2082*.nc",
    "/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2083*.nc"
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2080*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2081*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2082*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_0509_sugarcane/lnd/hist/NSSP534frc2esm_f19_tn14_0509_sugarcane.clm2.h0.2083*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_2009_sugarcane_restart2060/lnd/hist/NSSP534frc2esm_f19_tn14_2009_sugarcane_restart2060.clm2.h0.2060*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2040/lnd/hist/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2040.clm2.h0.2080*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2040/lnd/hist/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2040.clm2.h0.2080*.nc",
    #"/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2060/lnd/hist/NSSP534frc2esm_f19_tn14_2109_sugarcane_emission2060.clm2.h0.2080*.nc"
    # Add more file paths here
]

sum_NPP_def_GtC_values = []

for file_path in file_paths:
    list_files_def = glob.glob(file_path)
    list_files_def = sorted(list_files_def)
    datasets_def = []

    for i in range(len(list_files_def)):
        dset_case_def = xr.open_dataset(list_files_def[i], decode_times=False)
        NPP_def = dset_case_def['NPP']
        datasets_def.append(NPP_def)

    combined_dataset_def = xr.concat(datasets_def, dim='time')
    sum_NPP_def_GtC = combined_dataset_def.sum(['time']) * 86400 * 30.5 * (1e-15) * dset_case_def['area'] * (1e6)
    sum_NPP_def_GtC_values.append(sum_NPP_def_GtC)

# Now, `sum_NPP_def_GtC_values` contains the separate sum values for each set of file paths
# You can loop through these values and create plots for each set of file paths
variables = ['sum_NPP_def_GtC_2080', 'sum_NPP_def_GtC_2081','sum_NPP_def_GtC_2082','sum_NPP_def_GtC_2083','sum_NPP_cane_GtC_2080','sum_NPP_cane_GtC_2081','sum_NPP_cane_GtC_2082',
             'sum_NPP_cane_GtC_2083','sum_NPP_def_GtC_2060', 'sum_NPP_def_GtC_2040','sum_NPP_def_GtC_2080with2040emis', 'sum_NPP_def_GtC_2080with2060emis']  # Add variable names as needed

for i, variable in enumerate(variables):
    data = sum_NPP_def_GtC_values[i]
    data = data.where(data != 0)
    fig, axs = plt.subplots(figsize=(8, 4), subplot_kw={'projection': ccrs.PlateCarree()})
    xr.set_options(cmap_sequential=plt.cm.YlGn, keep_attrs=True)
    cmap = plt.cm.get_cmap()
    cmap.set_under('gray')
    cmap = plt.get_cmap('jet',)
    data.plot(cmap=cmap,cbar_kwargs={'label': 'NPP(GtC/year)'},vmin=0,vmax=0.1)

    land = cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='none', facecolor='white')
    axs.add_feature(land, zorder=0)
    axs.coastlines(linewidth=1)
    axs.set_xticks(np.arange(-180, 181, 60), crs=ccrs.PlateCarree())
    lon_formatter = cticker.LongitudeFormatter()
    axs.xaxis.set_major_formatter(lon_formatter)
    axs.set_yticks(np.arange(-90, 91, 30), crs=ccrs.PlateCarree())
    lat_formatter = cticker.LatitudeFormatter()
    axs.yaxis.set_major_formatter(lat_formatter)
    axs.set_title(f'{variable}', fontsize=16, loc='center')
    annual_yield = (np.sum(data)).item()
    axs.text(0.8, -0.3, f"{annual_yield:.3f}", fontsize=10, color='k', ha='center', va='top', transform=plt.gca().transAxes)
    fig.patch.set_facecolor('white')
    plt.show()


ValueError: must supply at least one object to concatenate

##just to test the plotting techniques

In [3]:
variables = ['sum_NPP_def_GtC_2080']#,'sum_NPP_corn_GtC_2080', 'sum_NPP_cane_GtC_2080', 'sum_NPP_cane_GtC_2081', 'sum_NPP_cane_GtC_2082', 'sum_NPP_cane_GtC_2083'] #'NPP_sum_GtC_year_corn',

for variable in variables:
    # Access the data for the current variable
    data = globals()[variable]  # Assuming the variables are defined in the global scope
    data = data.where(data != 0)
    fig, axs = plt.subplots(figsize=(8, 4), subplot_kw={'projection': ccrs.PlateCarree()})
    xr.set_options(cmap_sequential=plt.cm.YlGn, keep_attrs=True)
    cmap = plt.cm.get_cmap()
    cmap.set_under('gray')
    cmap = plt.get_cmap('jet',)
    plot=data.plot(cmap=cmap,cbar_kwargs={'label': 'NPP(GtC/year)'})

    land = cfeature.NaturalEarthFeature('physical', 'land', '110m', edgecolor='none', facecolor='white')
    axs.add_feature(land, zorder=0)
    axs.coastlines(linewidth=1)
    axs.set_xticks(np.arange(-180, 181, 60), crs=ccrs.PlateCarree())
    lon_formatter = cticker.LongitudeFormatter()
    axs.xaxis.set_major_formatter(lon_formatter)
    axs.set_yticks(np.arange(-90, 91, 30), crs=ccrs.PlateCarree())
    lat_formatter = cticker.LatitudeFormatter()
    axs.yaxis.set_major_formatter(lat_formatter)
    axs.set_title(f'{variable}', fontsize=16, loc='center')
    annual_yield = (np.sum(data)).item()
    axs.text(0.8, -0.3, f"{annual_yield:.3f}", fontsize=10, color='k', ha='center', va='top', transform=plt.gca().transAxes)
    fig.patch.set_facecolor('white')
    plt.show()

KeyError: 'sum_NPP_def_GtC_2080'

##READ SINGLE FILE

file_paths = [
    "/cluster/work/users/a2021/archive/NSSP534frc2esm_f19_tn14_1008_withrestart/lnd/hist/NSSP534frc2esm_f19_tn14_1008_withrestart.clm2.h0.2080*.nc",
]

sum_NPP_def_GtC_values = []

for file_path in file_paths:
    list_files_def = glob.glob(file_path)
    list_files_def = sorted(list_files_def)
    datasets_def = []

    for i in range(len(list_files_def)):
        dset_case_def = xr.open_dataset(list_files_def[i], decode_times=False)
        NPP_def = dset_case_def['NPP']
        datasets_def.append(NPP_def)

    combined_dataset_def = xr.concat(datasets_def, dim='time')
    sum_NPP_def_GtC = combined_dataset_def.sum(['time']) * 86400 * 30 * (1e-15) * dset_case_def['area'] * (1e6)
    sum_NPP_def_GtC_values.append(sum_NPP_def_GtC)